In [ ]:
# pip install presidio-analyzer presidio-anonymizer
# python -m spacy download en_core_web_md
from openai import OpenAI
import pandas as pd
import time
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

# === User Configuration ===
NUM_ROWS_TO_PROCESS = 20  # 👈 Change this value to process a different number of rows
INPUT_FILE = "sample_input.xlsx"
OUTPUT_FILE = "sample_output.xlsx"
API_KEY = "" # 👈 Enter your key

# === Prompt Configuration ===
CLASSIFIER_PROMPT = """
You are a classifier. Return Y if the input describes a problem or dissatisfaction that needs internal action or resolution. 
Return N otherwise. Only respond with Y or N. Anything related to flights is not a complaint. 
Everything that relates to car accidents or program rules violation such as stipends is considered a complaint. 
It has to be a direct complaint against the service of the au pair agency. 
If the customer describes a situation where immigration asked for a letter from the Responsible Officer or Program Sponsor, this is a request, not a complaint. 
Don't return anything other than Y or N.
"""

REASON_PROMPT = """You are a complaint classifier for an au pair program. Given the input text, determine if the issue described falls into one of the following specific complaint categories. Return only the exact matching category from the list below. If the text does not fall into any of these categories, return an empty string.
Categories:
Host Family Not Cooperative Regarding Au Pair Participant's Educational Component
Host family Requesting/Requiring More Than 45 Hours per Week or 10 Hours Per Day
Host Family Requesting/Requiring Housekeeping Chores Beyond Those Directly Related to Childcare
Au Pair is Unwilling to Work Required Time
Au Pair Provides Inadequate Childcare
Au pair is Not Proficient in English
Au Pair Did Not Receive Required Training
Au Pair Does not Have Experience Needed by the Family
Issues Concerning Scheduling -- Time Off, Vacation, Daily Routine
Issues Regarding Money - Pay, Reimbursements for Damages, Calls
Issues Concerning Hygiene and Privacy
Issues Concerning Behavior and or Inappropriate Activity
Placement Location
Personality Differences
Au Pair's Driving Skills
Au Pair Medical Reasons
Family Illness/Death/Emergency
Participant Does Not Want to Be an Au Pair
Host Family Home Not Conducive to Cultural Exchange
Homesickness
Host Family Leaving the Program
Au Pair Inability to Follow Guidelines
Personal Reasons
Instructions:
- Output only one category if the complaint clearly fits into one.
- If more than one category applies, return the most relevant one.
- If none apply, return an empty string."""

REPORTER_PROMPT = """
You are a classifier that determines who reported a complaint in a cultural exchange program involving host families (HF) and au pairs (AP).
Analyze the provided text and return only one of the following options:
HF (if the complaint was reported by the host family)
AP (if the complaint was reported by the au pair)
Leave it blank if it is not possible to determine who reported it from the context.
Guidelines:
If the text contains phrases like “HM informed us” or “HD stated that”, this means Host Family reported it → return HF.
If the text refers to the au pair directly contacting the agency or expressing dissatisfaction, return AP.
If the context is ambiguous or reported by a third party without clear indication, leave the result blank.
Example Input/Output:
Input: "HM informed us that the au pair often comes home past curfew." Output: HF
Input: "The au pair expressed discomfort with the family’s rules." Output: AP
"""

# === Initialize Presidio ===
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

def anonymize_with_presidio(text):
    if pd.isna(text):
        return ""
    text = str(text)
    results = analyzer.analyze(text=text, language="en")
    anonymized = anonymizer.anonymize(text=text, analyzer_results=results)
    return anonymized.text

# === Initialize OpenAI Client ===
client = OpenAI(api_key=API_KEY)
print("✅ Client ready")

# === Load and Preprocess Data ===
df = pd.read_excel(INPUT_FILE)
print("✅ Data loaded")

# Apply anonymization
df['Anonymized_Description'] = df['Description'].apply(anonymize_with_presidio)
df['Anonymized_Subject'] = df['Subject'].apply(anonymize_with_presidio)
print("✅ Subject and Description anonymized")

# Limit rows based on user-defined variable
df = df.head(NUM_ROWS_TO_PROCESS)
df['Complaint?'] = ""
df['Complaint_Reason'] = ""  # Column for storing the complaint reason
df['Reported by HF or AP'] = ""  # New column for storing who reported the complaint

# === Define Classifiers ===
def classify_complaint(subject, description):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": CLASSIFIER_PROMPT
                },
                {
                    "role": "user",
                    "content": f"Is the following case a complaint?\nSubject: \"{subject}\"\nDescription: \"{description}\""
                }
            ],
            temperature=0
        )
        result = response.choices[0].message.content.strip()
        return result if result in ['Y', 'N'] else "Error"
    except Exception as e:
        return f"Error: {str(e)}"

def determine_complaint_reason(subject, description):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": REASON_PROMPT
                },
                {
                    "role": "user",
                    "content": f"Subject: \"{subject}\"\nDescription: \"{description}\""
                }
            ],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

def determine_complaint_reporter(subject, description):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": REPORTER_PROMPT
                },
                {
                    "role": "user",
                    "content": f"Subject: \"{subject}\"\nDescription: \"{description}\""
                }
            ],
            temperature=0
        )
        result = response.choices[0].message.content.strip()
        # Validate that we got one of the expected responses
        if result not in ["HF", "AP", "Other"]:
            return "Other"  # Default to "Other" if unexpected response
        return result
    except Exception as e:
        return f"Error: {str(e)}"

# === Run Classification ===
total_rows = len(df)
for idx, row in df.iterrows():
    print(f"🔄 Processing row {idx + 1}/{total_rows} ({(idx + 1)/total_rows*100:.1f}%)")
    subject = row.get('Anonymized_Subject', '')
    description = row.get('Anonymized_Description', '')
    
    # First, determine if it's a complaint
    complaint_result = classify_complaint(subject, description)
    df.at[idx, 'Complaint?'] = complaint_result
    
    # If it's a complaint (Y), determine the reason and who reported it
    if complaint_result == 'Y':
        print(f"  ➡️ Determining complaint details for row {idx + 1}")
        
        # Determine complaint reason
        reason = determine_complaint_reason(subject, description)
        df.at[idx, 'Complaint_Reason'] = reason
        time.sleep(1)  # API rate limiting
        
        # Determine who reported the complaint
        reporter = determine_complaint_reporter(subject, description)
        df.at[idx, 'Reported by HF or AP'] = reporter
        time.sleep(1)  # Additional delay for API rate limiting
    
    time.sleep(1)  # Respectful API rate limiting

# === Save Output ===
df.to_excel(OUTPUT_FILE, index=False)
print(f"✅ Done. Results saved to {OUTPUT_FILE}")